In [ ]:
from pathlib import Path

import ee
import eemont  # noqa: F401
import geemap
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from bisonlab.data import landsat_7_sr, landsat_8_sr, landsat_9_sr
from bisonlab.io import kml_to_geodataframe
from bisonlab.utils import mask_exclude

In [ ]:
ee.Authenticate()
ee.Initialize()

## Load Parcels and Masks

In [ ]:
# Set path to local data directory
local_data = Path().cwd().parent.parent.parent / "data" / "local"

In [ ]:
df_parcels = kml_to_geodataframe(local_data / "WRTBI-Morton Soil Types.kml")

df_parcels = df_parcels.rename(columns={"Name": "subsection", "layer": "parcel"})
df_parcels = df_parcels.drop(columns="Description")

# Union all "mask" polygons and create a separate mask dataframe
idx = df_parcels["parcel"] == "mask"
df_mask = df_parcels[idx].dissolve(by="parcel").reset_index()

# Drop "mask" from parcel dataframe
df_parcels = df_parcels.drop(index=idx.index[idx])

# Convert parcels dataframe to ee.featureCollection
parcels = geemap.geopandas_to_ee(df_parcels)
mask = geemap.geopandas_to_ee(df_mask)

In [ ]:
df_parcels

In [ ]:
m = geemap.Map()
m.add_basemap(basemap="SATELLITE")
m.addLayer(parcels, {"color": "red"}, "Parcels")
m.centerObject(parcels, 15)
m

## Retrieve Landsat Data

In [ ]:
from tqdm.auto import tqdm

### Save data locally

In [ ]:
source_config = {
    "landsat7": {"source": landsat_7_sr, "years": range(2000, 2023)},
    "landsat8": {"source": landsat_8_sr, "years": range(2013, 2023)},
    "landsat9": {"source": landsat_9_sr, "years": range(2021, 2023)},
}

In [ ]:
for source, config in source_config.items():
    print(source, config["years"])

    for year in tqdm(config["years"]):

        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"

        fc = config["source"](
            parcels, start_date, end_date, cloud_prob_thresh=100
        ).spectralIndices(["NDVI", "EVI"])

        # Apply mask to each image in the collection
        fc_masked = fc.map(lambda img: mask_exclude(img, mask))

        # Aggregate data by parcel
        ts = fc_masked.getTimeSeriesByRegions(
            reducer=[
                ee.Reducer.mean(),
                ee.Reducer.stdDev(),
                ee.Reducer.min(),
                ee.Reducer.max(),
                ee.Reducer.count(),
            ],
            collection=parcels,
            bands=["NDVI", "EVI"],
            scale=30,
            dateColumn="date",
            naValue=None,
        )

        df = geemap.ee_to_pandas(ts)
        df["name"] = df["parcel"] + " - " + df["subsection"]

        filepath = local_data / f"long_time_series_{source}_{year}.parquet"
        df.to_parquet(filepath, index=None)

### Read local data

In [ ]:
df_list = []

for source, config in source_config.items():
    print(source, config["years"])

    for year in tqdm(config["years"]):
        filepath = local_data / f"long_time_series_{source}_{year}.parquet"
        df = pd.read_parquet(filepath)
        df["source"] = source
        df_list.append(df)
df = pd.concat(df_list)

In [ ]:
# Drop NaN
df = df.dropna(subset=["NDVI", "EVI"])

In [ ]:
# melt to long form table
df_long = df.melt(
    id_vars=["date", "source", "subsection", "parcel", "name", "reducer"],
    value_vars=["NDVI", "EVI"],
)

In [ ]:
# Drop values >1 or <-1 as these are erroneous
df_long = df_long.loc[(df_long["value"] <= 1) & (df_long["value"] >= -1)]

### Plot time series of parcel means

In [ ]:
df_plot = df_long.loc[
    df_long["date"].dt.year.eq(2022)
    & (
        df_long[["parcel", "subsection"]]
        .eq(["Shoshone Tribe", "High Steppe Unirrigated 1"])
        .all(axis=1)
        | df_long[["parcel", "subsection"]]
        .eq(["Hellyer Tribal Lease", "River Bottom 1"])
        .all(axis=1)
    )
    & df_long["reducer"].eq("mean")
]

g = sns.relplot(
    data=df_plot,
    x="date",
    y="value",
    hue="parcel",
    style="source",
    col="variable",
    col_wrap=1,
    kind="scatter",
    height=4,
    aspect=3,
    facet_kws=dict(sharey=False),
)

for ax in g.axes:
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_minor_locator(mdates.MonthLocator())

In [ ]:
df_plot = df_long.loc[df_long["reducer"].eq("mean")]

g = sns.relplot(
    data=df_plot,
    x="date",
    y="value",
    hue="variable",
    style="source",
    col="name",
    col_wrap=1,
    kind="scatter",
    height=4,
    aspect=3,
    facet_kws=dict(sharey=True, ylim=(-0.6, 1)),
)

for ax in g.axes:
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_minor_locator(mdates.MonthLocator())

plt.savefig(local_data / "long_time_series.png", dpi=240)

### Plot single parcel with errorbars

In [ ]:
ids = pd.IndexSlice

In [ ]:
ndvi = df.pivot(index=["date", "name"], columns="reducer", values="NDVI")

name = "Shoshone Tribe - High Steppe Unirrigated 1"
a = ndvi.loc[ids[:, name], :].reset_index(level="name", drop=True).sort_values("date")

fig, ax = plt.subplots(figsize=(15, 4))
ax.errorbar(
    x=a.index,
    y=a["mean"],
    yerr=a["stdDev"],
    fmt=".",
    linewidth=0,
    elinewidth=0.5,
    color="k",
    capthick=0.5,
    capsize=1,
)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_minor_locator(mdates.MonthLocator())
ax.set_ylabel("NDVI")
ax.set_title(name + " : NDVI")
fig.tight_layout()
fig.savefig(local_data / f"{name}_ndvi.png", dpi=240)

### Plot Monthly Mean Per Year

In [ ]:
b = a.copy()
b["month"] = b.index.month
b["year"] = b.index.year

c = b.pivot_table(index="year", columns="month", values="mean", aggfunc="mean")

fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(c, ax=ax, cmap="viridis")
ax.set_title(f"{name} : NDVI")
fig.savefig(local_data / f"{name}_ndvi_monthly.png", dpi=240);